In [11]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local[*]') \
        .appName('Basics') \
        .getOrCreate()

print(spark.version)

3.5.4


In [14]:
df_2018 = spark.read.csv(f"C:/Users/gyalm/OneDrive/Desktop/churn_prediction/churn_prediction/data/week2_for_teacher_demo.csv.gz", inferSchema=True, header=True)
df_2018.show(5)

+----------------+-----------+--------------------+------+-------+--------+------+-----+---------+-----------+------+-------------+
|transaction_date|member_type|           member_id|gender|product|quantity|amount|  age|card_type|branch_name|region|category_name|
+----------------+-----------+--------------------+------+-------+--------+------+-----+---------+-----------+------+-------------+
|      2018-01-17|     member|ffc9e419-cfda-11e...|FEMALE|    420|       2|  5300|50-55|  REGULAR|    D_Store|     2|          PET|
|      2018-04-10|     member|ff39d825-cfda-11e...|  MALE|     69|       4|  7100|40-45|  REGULAR|    D_Store|     2|     CLEANERS|
|      2018-02-19|     member|ff90d1f8-cfda-11e...|FEMALE|   1015|      18|  8100|40-45|  REGULAR|    J_Store|     1|          PET|
|      2018-01-27|     member|ffd5caf9-cfda-11e...|  MALE|     69|      11|  7100|18-25|  REGULAR|    H_Store|     4|     CLEANERS|
|      2018-05-07|     member|ff92f50e-cfda-11e...|FEMALE|   3456|      13| 

In [15]:
df_2018.createOrReplaceTempView("df_2018_view")

In [16]:
spark.sql('''SELECT * FROM df_2018_view LIMIT 5''').show()

+----------------+-----------+--------------------+------+-------+--------+------+-----+---------+-----------+------+-------------+
|transaction_date|member_type|           member_id|gender|product|quantity|amount|  age|card_type|branch_name|region|category_name|
+----------------+-----------+--------------------+------+-------+--------+------+-----+---------+-----------+------+-------------+
|      2018-01-17|     member|ffc9e419-cfda-11e...|FEMALE|    420|       2|  5300|50-55|  REGULAR|    D_Store|     2|          PET|
|      2018-04-10|     member|ff39d825-cfda-11e...|  MALE|     69|       4|  7100|40-45|  REGULAR|    D_Store|     2|     CLEANERS|
|      2018-02-19|     member|ff90d1f8-cfda-11e...|FEMALE|   1015|      18|  8100|40-45|  REGULAR|    J_Store|     1|          PET|
|      2018-01-27|     member|ffd5caf9-cfda-11e...|  MALE|     69|      11|  7100|18-25|  REGULAR|    H_Store|     4|     CLEANERS|
|      2018-05-07|     member|ff92f50e-cfda-11e...|FEMALE|   3456|      13| 

In [17]:
spark.sql('''SELECT COUNT(DISTINCT member_id) AS MemberId FROM df_2018_view''').show()

+--------+
|MemberId|
+--------+
|  302262|
+--------+



In [18]:
spark.sql('''SELECT COUNT(DISTINCT member_id) AS MemberId FROM df_2018_view WHERE transaction_date LIKE "%2018-04%"''').show()

+--------+
|MemberId|
+--------+
|   67717|
+--------+



In [29]:
# month_txn_window = [
#     ('2018-01-01', '2018-01-31', 'pm1_total_txn'),
#     ('2018-02-01', '2018-02-28', 'pm2_total_txn'),
#     ('2018-03-01', '2018-03-31', 'pm3_total_txn'),
#     ('2018-04-01', '2018-04-30', 'pm4_total_txn'),
#     ('2018-05-01', '2018-05-31', 'pm5_total_txn'),
#     ('2018-06-01', '2018-06-30', 'pm6_total_txn')]

# sum_txn_window = ",\n        ".join([
#     f"SUM(CASE WHEN transaction_date BETWEEN '{start_date}' AND '{end_date}' THEN quantity ELSE 0 END) AS {past_month}"
# for start_date, end_date, past_month in month_txn_window
# ])

# spark.sql(f'''
# WITH
# aggregated_customers AS (
#     SELECT DISTINCT member_id
#     FROM df_2018_view
#     WHERE transaction_date BETWEEN '{month_txn_window[::len(month_txn_window)][0][0]}' AND '{month_txn_window[::-1][0][1]}'
# ),

# aggregated_customers_with_past_txns AS (
#     SELECT DISTINCT
#         a.member_id,
#         b.quantity,
#         b.transaction_date
#     FROM
#         aggregated_customers AS a
#     LEFT JOIN
#         df_2018_view AS b
#     ON
#         a.member_id = b.member_id
#     WHERE
#         b.transaction_date BETWEEN '{month_txn_window[::len(month_txn_window)][0][0]}' AND '{month_txn_window[::-1][0][1]}'
# ),

# aggregated_total_txns_per_month AS (
#     SELECT DISTINCT
#         member_id,
#         {sum_txn_window}
#     FROM
#         aggregated_customers_with_past_txns
#     GROUP BY
#         member_id
# )

# SELECT DISTINCT
#     member_id,
#     {", ".join(list(map(lambda x: x[2], month_txn_window)))},
#     CASE 
#         WHEN { " AND ".join(list(map(lambda x: f"{x[2]} = 0", month_txn_window))) } THEN 'Churned'
#         ELSE 'Active'
#     END AS churn_status
# FROM
#     aggregated_total_txns_per_month
# LIMIT 
# 20
# ''').show()

In [19]:
spark.sql('''
WITH customers_in_april AS (
  SELECT DISTINCT member_id
  FROM df_2018_view
  WHERE transaction_date BETWEEN '2018-04-01' AND '2018-04-30'
), 
customers_in_april_joined_with_past_3_months_txn AS (
  SELECT
    a.member_id, 
    b.quantity, 
    b.transaction_date
  FROM
    customers_in_april AS a
  LEFT JOIN
    df_2018_view AS b
  ON
    a.member_id = b.member_id
  WHERE
    b.transaction_date BETWEEN '2018-01-01' AND '2018-03-31'
), 
aggregated_total_txns_in_april AS (
  SELECT
    member_id, 
    SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-01-31' THEN quantity ELSE 0 END) AS pm3_total_txn, 
    SUM(CASE WHEN transaction_date BETWEEN '2018-02-01' AND '2018-02-28' THEN quantity ELSE 0 END) AS pm2_total_txn, 
    SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-03-31' THEN quantity ELSE 0 END) AS pm1_total_txn,
    '2018-04-01' AS PARTITION_MONTH
  FROM
    customers_in_april_joined_with_past_3_months_txn
  GROUP BY
    member_id
),
customers_in_may AS (
  SELECT DISTINCT member_id
  FROM df_2018_view
  WHERE transaction_date BETWEEN '2018-05-01' AND '2018-05-31'
), 
customers_in_may_joined_with_past_3_months_txn AS (
  SELECT
    a.member_id, 
    b.quantity, 
    b.transaction_date
  FROM
    customers_in_may AS a
  LEFT JOIN
    df_2018_view AS b
  ON
    a.member_id = b.member_id
  WHERE
    b.transaction_date BETWEEN '2018-02-01' AND '2018-04-30'
), 
aggregated_total_txns_in_may AS (
  SELECT
    member_id, 
    SUM(CASE WHEN transaction_date BETWEEN '2018-02-01' AND '2018-02-28' THEN quantity ELSE 0 END) AS pm3_total_txn, 
    SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-03-31' THEN quantity ELSE 0 END) AS pm2_total_txn, 
    SUM(CASE WHEN transaction_date BETWEEN '2018-04-01' AND '2018-04-30' THEN quantity ELSE 0 END) AS pm1_total_txn,
    '2018-05-01' AS PARTITION_MONTH
  FROM
    customers_in_may_joined_with_past_3_months_txn
  GROUP BY
    member_id
),
customers_in_june AS (
  SELECT DISTINCT member_id
  FROM df_2018_view
  WHERE transaction_date BETWEEN '2018-06-01' AND '2018-06-30'
), 
customers_in_june_joined_with_past_3_months_txn AS (
  SELECT
    a.member_id, 
    b.quantity, 
    b.transaction_date
  FROM
    customers_in_june AS a
  LEFT JOIN
    df_2018_view AS b
  ON
    a.member_id = b.member_id
  WHERE
    b.transaction_date BETWEEN '2018-03-01' AND '2018-05-31'
), 
aggregated_total_txns_in_june AS (
  SELECT
    member_id, 
    SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-03-31' THEN quantity ELSE 0 END) AS pm3_total_txn, 
    SUM(CASE WHEN transaction_date BETWEEN '2018-04-01' AND '2018-04-30' THEN quantity ELSE 0 END) AS pm2_total_txn, 
    SUM(CASE WHEN transaction_date BETWEEN '2018-05-01' AND '2018-05-31' THEN quantity ELSE 0 END) AS pm1_total_txn,
    '2018-06-01' AS PARTITION_MONTH
  FROM
    customers_in_june_joined_with_past_3_months_txn
  GROUP BY
    member_id
),
union_all_aggregated_txns AS (
  SELECT * FROM aggregated_total_txns_in_april
  UNION ALL
  SELECT * FROM aggregated_total_txns_in_may
  UNION ALL
  SELECT * FROM aggregated_total_txns_in_june
)
SELECT
    PARTITION_MONTH,
    COUNT(member_id) AS total_member_id
FROM
    union_all_aggregated_txns
GROUP BY
    PARTITION_MONTH
ORDER BY
    PARTITION_MONTH
''').show()

+---------------+---------------+
|PARTITION_MONTH|total_member_id|
+---------------+---------------+
|     2018-04-01|          28579|
|     2018-05-01|          41565|
|     2018-06-01|          45672|
+---------------+---------------+

